In [1]:
import tensorflow as tf
from model_v2 import LanguageModel
import json
import numpy as np
from utils import batchify, get_batch
import time

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # pylint: disable=no-member
sess = tf.Session(config=config)

In [4]:
with open('106/checkpoints/model_configs.json', 'r') as inp:
    params = json.load(inp)

In [5]:
model = LanguageModel(**params, is_training=False)

In [6]:
model.build_model()
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection('LSTM_SAVED_STATE')])
sess.run(tf.global_variables_initializer())

In [7]:
saver.restore(sess, '106/checkpoints/test/model.cpkt-54732')

INFO:tensorflow:Restoring parameters from 106/checkpoints/test/model.cpkt-54732


In [8]:
with open('wiki_bak/new/valid_char.npy', 'rb') as inp:
    test_char = np.load(inp)
with open('wiki_bak/new/valid_word.npy', 'rb') as inp:
    test_word = np.load(inp)
print(len(test_word), len(test_char))

1999242 1999242


In [9]:
test_word = batchify(test_word, 54).T
test_char = batchify(test_char, 54).T

In [10]:
fw_iy = tf.placeholder(dtype=tf.int32, shape=[None, None], name='fw_y')
bw_iy = tf.placeholder(dtype=tf.int32, shape=[None, None], name='bw_y')
test_loss = 0.5 * tf.add(
    tf.contrib.seq2seq.sequence_loss(
        logits=model.fw_model['decoder'],
        targets=fw_iy,
        weights=model.seq_masks,
        average_across_timesteps=True,
        average_across_batch=True),
    tf.contrib.seq2seq.sequence_loss(
        logits=model.bw_model['decoder'],
        targets=bw_iy,
        weights=model.seq_masks,
        average_across_timesteps=True,
        average_across_batch=True),
    name='test_loss'
)
fw_acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(model.fw_model['decoder'], axis=-1, output_type=tf.int32), fw_iy)))
bw_acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(model.bw_model['decoder'], axis=-1, output_type=tf.int32), bw_iy)))

In [11]:
def evaluate_step(test_word, test_char, bptt):
    start_time = time.time()
    total_loss = 0
    total_fa = 0
    total_ba = 0
    step = None
    for i in range(0, len(test_word), bptt):
        (fw_x, fw_cl, fw_y), (bw_x, bw_cl, bw_y) = get_batch(test_word, test_char, bptt, i, evaluate=True)
        loss, fa, ba = sess.run(
            [test_loss, fw_acc, bw_acc],
            feed_dict={
                model.fw_inputs: fw_x, model.fw_char_lens: fw_cl,
                model.bw_inputs: bw_x, model.bw_char_lens: bw_cl,
                fw_iy: fw_y,
                bw_iy: bw_y,
                model.seq_lens: [fw_y.shape[0]] * fw_y.shape[1],
                model.reset_state: i == 0
            }
        )
        total_loss += loss * len(fw_y)
        total_fa += fa * len(fw_y)
        total_ba += ba * len(fw_y)
        print("Evaluate loss {:05.5f}, forward acc {:05.5f}, backward acc {:05.5f}, time {}".format(loss, fa, ba, time.time()-start_time))
    total_loss /= len(test_word)
    print("Evaluate total loss {:05.5f}, forward acc {:05.5f}, backward acc {:05.5f}, time {}".format(loss, fa, ba, time.time()-start_time))

In [12]:
evaluate_step(test_word, test_char, 54)

Evaluate loss 3.32866, forward acc 0.42284, backward acc 0.41427, time 0.8235812187194824
Evaluate loss 3.09026, forward acc 0.44033, backward acc 0.45988, time 1.2447199821472168
Evaluate loss 3.11162, forward acc 0.44307, backward acc 0.47051, time 1.6230638027191162
Evaluate loss 3.09236, forward acc 0.45576, backward acc 0.44684, time 2.0557985305786133
Evaluate loss 3.05523, forward acc 0.44890, backward acc 0.47154, time 2.4412009716033936
Evaluate loss 3.01275, forward acc 0.46125, backward acc 0.46262, time 2.864179849624634
Evaluate loss 3.09001, forward acc 0.44067, backward acc 0.45610, time 3.2415223121643066
Evaluate loss 3.08264, forward acc 0.44856, backward acc 0.45062, time 3.62646746635437
Evaluate loss 3.07849, forward acc 0.44650, backward acc 0.45130, time 4.006320238113403
Evaluate loss 3.00546, forward acc 0.44753, backward acc 0.46742, time 4.389984369277954
Evaluate loss 2.84564, forward acc 0.48388, backward acc 0.47531, time 4.775115966796875
Evaluate loss 2.

Evaluate loss 2.94690, forward acc 0.50343, backward acc 0.45096, time 36.41235828399658
Evaluate loss 2.89623, forward acc 0.50171, backward acc 0.45508, time 36.79156231880188
Evaluate loss 2.89506, forward acc 0.49931, backward acc 0.45953, time 37.1789333820343
Evaluate loss 2.78239, forward acc 0.50754, backward acc 0.47497, time 37.56322717666626
Evaluate loss 2.93224, forward acc 0.47805, backward acc 0.45473, time 37.983458280563354
Evaluate loss 2.84296, forward acc 0.49383, backward acc 0.46811, time 38.36478662490845
Evaluate loss 2.89943, forward acc 0.47668, backward acc 0.46742, time 38.74715876579285
Evaluate loss 2.78182, forward acc 0.49280, backward acc 0.50069, time 39.1270866394043
Evaluate loss 2.77540, forward acc 0.47428, backward acc 0.49348, time 39.51135563850403
Evaluate loss 2.95589, forward acc 0.48663, backward acc 0.46536, time 39.89085674285889
Evaluate loss 2.91620, forward acc 0.48491, backward acc 0.47977, time 40.27288103103638
Evaluate loss 3.00587,

Evaluate loss 2.92058, forward acc 0.48354, backward acc 0.45130, time 71.82362413406372
Evaluate loss 2.97168, forward acc 0.47840, backward acc 0.46159, time 72.20628905296326
Evaluate loss 2.93862, forward acc 0.47737, backward acc 0.45713, time 72.58823132514954
Evaluate loss 2.96942, forward acc 0.48834, backward acc 0.42695, time 72.97173571586609
Evaluate loss 3.03622, forward acc 0.45645, backward acc 0.44925, time 73.35527157783508
Evaluate loss 2.93104, forward acc 0.48251, backward acc 0.46776, time 73.7399570941925
Evaluate loss 2.97320, forward acc 0.47222, backward acc 0.45713, time 74.12139058113098
Evaluate loss 3.04777, forward acc 0.45988, backward acc 0.44376, time 74.50819563865662
Evaluate loss 3.00910, forward acc 0.46091, backward acc 0.46228, time 74.89127922058105
Evaluate loss 3.01638, forward acc 0.46948, backward acc 0.45370, time 75.27513337135315
Evaluate loss 2.93401, forward acc 0.47599, backward acc 0.46296, time 75.65696144104004
Evaluate loss 2.88500,

Evaluate loss 2.87268, forward acc 0.48080, backward acc 0.46091, time 107.55860018730164
Evaluate loss 3.01313, forward acc 0.47428, backward acc 0.45165, time 107.94583106040955
Evaluate loss 2.86745, forward acc 0.49726, backward acc 0.45713, time 108.3352701663971
Evaluate loss 2.95834, forward acc 0.49794, backward acc 0.44890, time 108.72006750106812
Evaluate loss 2.88235, forward acc 0.49383, backward acc 0.46605, time 109.10490298271179
Evaluate loss 2.94190, forward acc 0.46948, backward acc 0.47908, time 109.48431968688965
Evaluate loss 2.88418, forward acc 0.47359, backward acc 0.48731, time 109.86745500564575
Evaluate loss 2.88033, forward acc 0.48251, backward acc 0.45953, time 110.2527825832367
Evaluate loss 2.92897, forward acc 0.44856, backward acc 0.47634, time 110.64034509658813
Evaluate loss 2.96100, forward acc 0.47119, backward acc 0.46228, time 111.02484512329102
Evaluate loss 3.04577, forward acc 0.46742, backward acc 0.43416, time 111.40831160545349
Evaluate los

Evaluate loss 3.03694, forward acc 0.46571, backward acc 0.45096, time 142.92456197738647
Evaluate loss 3.03559, forward acc 0.47051, backward acc 0.45988, time 143.30602407455444
Evaluate loss 2.94219, forward acc 0.46811, backward acc 0.46708, time 143.68933367729187
Evaluate loss 2.93717, forward acc 0.47702, backward acc 0.46502, time 144.06864976882935
Evaluate loss 2.91067, forward acc 0.47154, backward acc 0.49074, time 144.46052384376526
Evaluate loss 2.91879, forward acc 0.46674, backward acc 0.48594, time 144.84036898612976
Evaluate loss 2.99079, forward acc 0.46708, backward acc 0.45679, time 145.22480845451355
Evaluate loss 2.93660, forward acc 0.47325, backward acc 0.47051, time 145.60768604278564
Evaluate loss 2.90972, forward acc 0.48080, backward acc 0.47222, time 145.9991593360901
Evaluate loss 2.87715, forward acc 0.48971, backward acc 0.46433, time 146.38206315040588
Evaluate loss 2.94397, forward acc 0.47359, backward acc 0.46056, time 146.76386404037476
Evaluate lo

Evaluate loss 3.04269, forward acc 0.45508, backward acc 0.42455, time 178.2434492111206
Evaluate loss 3.00235, forward acc 0.45473, backward acc 0.43896, time 178.6266839504242
Evaluate loss 2.99355, forward acc 0.45508, backward acc 0.45473, time 179.00532960891724
Evaluate loss 3.00195, forward acc 0.46331, backward acc 0.44136, time 179.39756226539612
Evaluate loss 3.09587, forward acc 0.45679, backward acc 0.43381, time 179.78181719779968
Evaluate loss 3.00521, forward acc 0.44170, backward acc 0.47257, time 180.16697192192078
Evaluate loss 3.00917, forward acc 0.45679, backward acc 0.46056, time 180.5500795841217
Evaluate loss 3.04026, forward acc 0.46125, backward acc 0.46091, time 180.93517303466797
Evaluate loss 2.98060, forward acc 0.47462, backward acc 0.44822, time 181.3177900314331
Evaluate loss 2.99124, forward acc 0.46399, backward acc 0.46159, time 181.70117926597595
Evaluate loss 2.93529, forward acc 0.45816, backward acc 0.47565, time 182.08139634132385
Evaluate loss 

Evaluate loss 2.88601, forward acc 0.49348, backward acc 0.45165, time 213.60958242416382
Evaluate loss 2.86693, forward acc 0.46674, backward acc 0.48971, time 213.99967527389526
Evaluate loss 2.91475, forward acc 0.47840, backward acc 0.47805, time 214.38550925254822
Evaluate loss 2.90674, forward acc 0.46845, backward acc 0.47291, time 214.7688853740692
Evaluate loss 2.83764, forward acc 0.50343, backward acc 0.45165, time 215.15178060531616
Evaluate loss 2.82612, forward acc 0.47565, backward acc 0.47737, time 215.5355076789856
Evaluate loss 2.89946, forward acc 0.49897, backward acc 0.44582, time 215.9212989807129
Evaluate loss 2.79667, forward acc 0.50652, backward acc 0.48560, time 216.30387902259827
Evaluate loss 2.91287, forward acc 0.49246, backward acc 0.44684, time 216.68067526817322
Evaluate loss 2.95456, forward acc 0.49657, backward acc 0.45062, time 217.0657742023468
Evaluate loss 2.84079, forward acc 0.49211, backward acc 0.46948, time 217.44845581054688
Evaluate loss 

Evaluate loss 2.84316, forward acc 0.49451, backward acc 0.47668, time 249.15613555908203
Evaluate loss 2.80495, forward acc 0.48560, backward acc 0.49314, time 249.54039335250854
Evaluate loss 2.88085, forward acc 0.47805, backward acc 0.47599, time 249.92840886116028
Evaluate loss 2.91244, forward acc 0.45267, backward acc 0.47359, time 250.30830073356628
Evaluate loss 2.90876, forward acc 0.46605, backward acc 0.49074, time 250.69348216056824
Evaluate loss 2.91804, forward acc 0.44650, backward acc 0.48868, time 251.0745084285736
Evaluate loss 2.94800, forward acc 0.46262, backward acc 0.47942, time 251.4538812637329
Evaluate loss 2.94118, forward acc 0.46433, backward acc 0.46605, time 251.83640336990356
Evaluate loss 2.87357, forward acc 0.46982, backward acc 0.46331, time 252.22366189956665
Evaluate loss 3.05817, forward acc 0.45679, backward acc 0.44410, time 252.6097490787506
Evaluate loss 2.99850, forward acc 0.46742, backward acc 0.45953, time 252.99594855308533
Evaluate loss